In [20]:
'''
this notebook will sequence images by minimizing total error/displacement
'''

'\nthis notebook will sequence images by minimizing total error/displacement\n'

In [11]:
import glob
import cv2
import matplotlib.pyplot as plt
from skimage.io import imread
import numpy as np
import pandas as pd

%matplotlib inline


In [7]:
ls_paths = glob.glob('0/*')[:20]
ls_images = map(imread, ls_paths)
len(ls_images)

20

In [8]:
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [9]:
src = []
dst = []
delta = []

for i in range(len(ls_images)):
    
    for j in range(len(ls_images)):
        
        old_image = ls_images[i]
        new_image = ls_images[j]
        old_gray = cv2.cvtColor(old_image, cv2.COLOR_BGR2GRAY)
        p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
        frame_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        sum(err)
        
        src.append(ls_paths[i])
        dst.append(ls_paths[j])
        delta.append(sum(err)[0])

In [15]:
# concat into dataframe for pairwise sorting

a = {'src': src}
b = {'dst': dst}
c = {'delta': delta}

data_x = pd.concat([pd.DataFrame(a), pd.DataFrame(b), pd.DataFrame(c)], axis = 1)
data_x.shape

(400, 3)

In [16]:
data_x['src'].unique()

array(['0/00035.jpg', '0/00036.jpg', '0/00039.jpg', '0/00079.jpg',
       '0/00103.jpg', '0/00136.jpg', '0/00139.jpg', '0/00143.jpg',
       '0/00148.jpg', '0/00153.jpg', '0/00159.jpg', '0/00180.jpg',
       '0/00211.jpg', '0/00216.jpg', '0/00224.jpg', '0/00241.jpg',
       '0/00247.jpg', '0/00248.jpg', '0/00260.jpg', '0/00263.jpg'], dtype=object)

In [17]:
def find_next(ls_sequences, data):
    '''
    function:
    ---------
    given a starting image, find the nearest neighbor by minimizing error'''
    
    idx = len(ls_sequences)
    prev = ls_sequences[idx  - 1]
    i = 1
    
    while True:

        
        next_image = data[data['src'] == prev].sort('delta').reset_index(drop = True)['dst'][i]

        if next_image not in ls_sequences_x:
            ls_sequences.append(next_image)
            break
        else:
            i += 1
    
    return ls_sequences

In [18]:
ls_sequences_x = ['0/00241.jpg'] # choose starting point!

# append to list until set is exhausted
while len(ls_sequences_x) < len(data['src'].unique()):
    find_next(ls_sequences_x, data_x)

20

In [19]:
pd.DataFrame(ls_sequences_x).to_csv('jet_1.csv')